In [1]:
import numpy as np
import cv2
import joblib
import argparse
import time
import pickle
# from time import gmtime, strftime

In [2]:
def detect_face(img, faceCascade):
    faces = faceCascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(110, 110)
        #flags = cv2.CV_HAAR_SCALE_IMAGE
    )
    return faces

In [3]:
def calc_hist(img):
    histogram = [0] * 3
    for j in range(3):
        histr = cv2.calcHist([img], [j], None, [256], [0, 256])
        histr *= 255.0 / histr.max()
        histogram[j] = histr
    return np.array(histogram)

In [4]:
import time
if __name__ == "__main__":
    l=dict()
    c=0
    
    clf = None
    try:
        clf = pickle.load(open("./model.pkl",'rb'))
    except IOError as e:
        print ("Error loading model <"+"./model.pkl"+">: {0}".format(e.strerror))
        exit(0)


    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print ("Error opening camera")
        exit(0)

    w = 500
    h = 400
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, h)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, w)
   
     

    filePath = "./haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(filePath)

    sample_number = 3
    count = 0
    measures = np.zeros(sample_number, dtype=np.float)

    while True:
        
        ret, img_bgr = cap.read()
        if ret is False:
            print("Error grabbing frame from camera")
            break

        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

        faces = detect_face(img_gray, faceCascade)

        measures[count%sample_number]=0

        point = (0,0)
        for i, (x, y, w, h) in enumerate(faces):

            roi = img_bgr[y:y+h, x:x+w]
            c+=1
            img_ycrcb = cv2.cvtColor(roi, cv2.COLOR_BGR2YCR_CB)
            img_luv = cv2.cvtColor(roi, cv2.COLOR_BGR2LUV)

            ycrcb_hist = calc_hist(img_ycrcb)
            luv_hist = calc_hist(img_luv)

            feature_vector = np.append(ycrcb_hist.ravel(), luv_hist.ravel())
            feature_vector = feature_vector.reshape(1, len(feature_vector))

            prediction = clf.predict_proba(feature_vector)
            prob = prediction[0][1]

            measures[count % sample_number] = prob

            cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (255, 0, 0), 2)

            point = (x, y-5)

            print(measures, np.mean(measures))
            if 0 not in measures:
                text = "Real"
                if np.mean(measures) >= 0.90:
                    text = "Fake"
                    font = cv2.FONT_HERSHEY_TRIPLEX
                    cv2.putText(img=img_bgr, text=text, org=point, fontFace=font, fontScale=0.9, color=(0, 0, 255),
                                thickness=2, lineType=cv2.LINE_AA)
                else:
                    #cv2.imshow("original",roi)
                    #time.sleep(10)
                    l.update({c:roi})
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img=img_bgr, text=text, org=point, fontFace=font, fontScale=0.9,
                                color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

        count+=1
        cv2.imshow('img_rgb', img_bgr)

        key = cv2.waitKey(1)
        if key & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator ExtraTreeC

[0.         0.16205534 0.        ] 0.05401844532279315
[0.         0.16205534 0.26477273] 0.1422760210803689
[0.29554196 0.16205534 0.26477273] 0.24079000709435494
[0.29554196 0.46477273 0.26477273] 0.3416958041958042
[0.29554196 0.46477273 0.35580534] 0.37204000709435486
[0.47330534 0.46477273 0.35580534] 0.431294466403162
[0.47330534 0.37330534 0.35580534] 0.4008053359683794
[0.47330534 0.37330534 0.43036339] 0.42565801951671517
[0.53709416 0.37330534 0.43036339] 0.44692095947530724
[0.53709416 0.45852273 0.43036339] 0.47532675657675655
[0.53709416 0.45852273 0.43852273] 0.47804653679653675
[0.30102273 0.45852273 0.43852273] 0.3993560606060606
[0.30102273 0.         0.1175    ] 0.13950757575757575
[0.68 0.   0.  ] 0.22666666666666666
[0.68       0.61477273 0.        ] 0.4315909090909091
[0.68       0.61477273 0.71477273] 0.6698484848484848
[0.70643939 0.61477273 0.71477273] 0.6786616161616162
[0.70643939 0.71477273 0.71477273] 0.7119949494949495
[0.70643939 0.71477273 0.61477273] 0.6

[0.92727273 0.82102273 0.82727273] 0.8585227272727275
[0.92012987 0.82102273 0.82727273] 0.8561417748917751
[0.92012987 0.79375    0.82727273] 0.847050865800866
[0.92012987 0.79375    0.78660714] 0.833495670995671
[0.99       0.79375    0.78660714] 0.8567857142857142
[0.99       0.98375    0.78660714] 0.9201190476190476
[0.99       0.98375    0.81102273] 0.9282575757575757
[0.78375    0.98375    0.81102273] 0.8595075757575757
[0.78375    0.89       0.81102273] 0.8282575757575757
[0.78375    0.89       0.81727273] 0.8303409090909092
[0.88375    0.89       0.81727273] 0.8636742424242424
[0.88375    0.91102273 0.81727273] 0.8706818181818181
[0.88375    0.91102273 0.88375   ] 0.8928409090909092
[0.89102273 0.91102273 0.88375   ] 0.8952651515151514
[0.89102273 0.99375    0.88375   ] 0.9228409090909091
[0.89102273 0.99375    0.91102273] 0.9319318181818183
[0.89375    0.99375    0.91102273] 0.9328409090909092
[0.89375    0.98375    0.91102273] 0.9295075757575756
[0.89375 0.98375 0.86375] 0.91

In [18]:
cv2.imwrite('5.jpg', l[1])

True